### The battle of neighbours

Data regarding new yrok geographics is readily available for free on :  https://geo.nyu.edu/catalog/nyu_2451_34572

### Liabraries import

In [4]:
import numpy as np
import pandas as pd
import json
import requests

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # library to convert geolications into lattitude and longitude
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #library for reading maps

print('All required libraries are imported')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Pratik\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             co

In [8]:

ny_data= requests.get('https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json')
    
ny_data = ny_data.json()

ny_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [9]:
neighborhoods_data = ny_data['features']

In [10]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [11]:
columns = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']
df = pd.DataFrame(columns=columns)

In [12]:
loc=neighborhoods_data[0]
lat = loc['geometry']['coordinates'][0]
lon=loc['geometry']['coordinates'][1]


### Transform data in pandas dataframe

In [13]:

for data in neighborhoods_data:
    borough = data['properties']['borough']
    neighborhood_name = data['properties']['name']
    lat = data['geometry']['coordinates'][0]
    lon = data['geometry']['coordinates'][1]
    
    df = df.append({'Borough': borough,'Neighborhood': neighborhood_name,'Latitude': lat,'Longitude': lon}, ignore_index=True)

In [14]:
df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,-73.847201,40.894705
1,Bronx,Co-op City,-73.829939,40.874294
2,Bronx,Eastchester,-73.827806,40.887556
3,Bronx,Fieldston,-73.905643,40.895437
4,Bronx,Riverdale,-73.912585,40.890834


### let us check number of borough and neighbourhoods dataset has

In [15]:
print('total number of borough: {}'.format(len(df['Borough'].unique())))
print('total number of neighborhoods: {}'.format(df.shape[0]))

total number of borough: 5
total number of neighborhoods: 306


In [16]:
df.to_csv('NYC_GEO.csv',index=False)

### lets get lattitude and longitude of NY

In [17]:
geolocator = Nominatim(user_agent="Jupyter")
location = geolocator.geocode('New York City, NY')
latitude = location.latitude
longitude = location.longitude
print('The coordinate of New York City are {}, {}.'.format(latitude, longitude))

The coordinate of New York City are 40.7127281, -74.0060152.


### lets create map of NY locating neighbourhoods

In [31]:
ny_map = folium.Map(location=[latitude, longitude],zoom_start=10)

for lat, lon, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html=True)
    cir=folium.CircleMarker(location=[lat, lon],radius=5,Popup=label,color='#3186cc',fill=True, fill_color='#3186cc',fill_opacity=0.7,parse_html=False).add_to(ny_map)
    
ny_map

